In [1]:
import scipy.io as scio
import netCDF4 as nc
import numpy as np
from Load_data import Data_from_nc
import xarray as xr
import math
import gc


def get_data_from_nc(file, variable):
    file_obj = nc.Dataset(file)
    data = file_obj.variables[variable]
    var_data = np.array(data)
    var_data = var_data[:, :, :]
    var_data = np.squeeze(var_data)
    return var_data
    

def Apparent_temperature_calcu(TSA, RH, U10):
    T_c = TSA - 273.15
    T_f = T_c * 1.8 + 32
    e_s = np.exp(53.67957 - 6743.769 / TSA - 4.8451 * np.log(TSA)) * 100
    e_RH = (RH / 100) * e_s
    AT = T_c + 3.3 * e_RH / 1000 - 0.7 * U10 - 4
    return AT

def Heat_index_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_f = T_c * 1.8 + 32
    HI = -42.379 + 2.04901523 * T_f + 10.14333127 * RH - 0.22475541 * T_f * RH -6.83783 * 0.001 * np.square(T_f) - 5.481717 * 0.01 * np.square(RH) + 1.22874 * 0.001 * np.square(T_f) * RH + 8.5282 * 0.0001 * T_f * np.square(RH) - 1.99 * 0.000001 * np.square(T_f) * np.square(RH)
    return HI

def Humidex_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_f = T_c * 1.8 + 32
    e_s = np.exp(53.67957 - 6743.769 / TSA - 4.8451 * np.log(TSA)) * 100
    e_RH = (RH / 100) * e_s
    HUMIDEX = T_c + 5/9 * (e_RH / 100 - 10)
    return HUMIDEX

def Wet_bulb_temperature_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_w = T_c * np.arctan(0.151977 * np.sqrt(RH + 8.313659)) + np.arctan(T_c + RH) - np.arctan(RH - 1.676331) + 0.00391838 * np.power(RH, 3/2) * np.arctan(0.023101 * RH) - 4.68035
    return T_w

def Temperature_humidity_index_for_comfort_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_w = Wet_bulb_temperature_calcu(TSA, RH)
    THIC = 0.72 * T_w + 0.72 * T_c + 40.6
    return THIC

def Temperature_humidity_index_for_physiology_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_w = Wet_bulb_temperature_calcu(TSA, RH)
    THIP = 0.63 * T_w + 1.17 * T_c + 32
    return THIP

def Discomfort_index_calcu(TSA, RH):
    T_c = TSA - 273.15
    T_w = Wet_bulb_temperature_calcu(TSA, RH)
    DI = 0.5 * T_w + 0.5 * T_c
    return DI

def Swamp_temperature_calcu(TSA, RH, eta):
    T_c = TSA - 273.15
    T_w = Wet_bulb_temperature_calcu(TSA, RH)
    T_s = T_c - eta / 100 * (T_c - T_w)
    return T_s


data_surface = Data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/surfdata_irrigation_method.nc')   #load the data
data_irrigation_method = data_surface.load_variable('irrigation_method')
data_lon = data_surface.load_variable('LONGXY')
data_lon = data_lon[0,:]
data_lat = data_surface.load_variable('LATIXY')
data_lat = data_lat[:,0]
data_lat = data_lat[:]

# Here we calculate the metrics based on the NorESM. For CESM2 and CESM2_gw, adjustments are needed
for exp in ['IRR01', 'IRR02', 'IRR03', 'NOI01', 'NOI02', 'NOI03']:
    for period in ["1901_1910","1911_1920","1921_1930", "1931_1940","1941_1950","1951_1960","1961_1970","1971_1980","1981_1984", "1985_1994","1995_2004","2005_2014"]:
        TSA_daily = get_data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/' + exp + '//3_hourly_mean/TSA/NorESM_' + exp + '_TSA_' + period + '.nc', 'TSA')
        RH2M_daily = get_data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/' + exp + '//3_hourly_mean/RH2M/NorESM_' + exp + '_RH2M_' + period + '.nc', 'RH2M')
        U10_daily = get_data_from_nc('/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/' + exp + '//3_hourly_mean/U10/NorESM_' + exp + '_U10_' + period + '.nc', 'U10')
        

        TSA_daily[TSA_daily > 1000] = np.nan
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/TSA_" + \
                     period + "_3_hourly_" + exp + "_1.mat", {'TSA': TSA_daily[:7300, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/TSA_" + \
                     period + "_3_hourly_" + exp + "_2.mat", {'TSA': TSA_daily[7300: 14600, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/TSA_" + \
                     period + "_3_hourly_" + exp + "_3.mat", {'TSA': TSA_daily[14600: 21900, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/TSA_" + \
                     period + "_3_hourly_" + exp + "_4.mat", {'TSA': TSA_daily[21900:, :, :]})

        
        AT_daily = Apparent_temperature_calcu(TSA_daily, RH2M_daily, U10_daily)
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/AT_" + \
                     period + "_3_hourly_" + exp + "_1.mat", {'AT': AT_daily[:7300, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/AT_" + \
                     period + "_3_hourly_" + exp + "_2.mat", {'AT': AT_daily[7300: 14600, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/AT_" + \
                     period + "_3_hourly_" + exp + "_3.mat", {'AT': AT_daily[14600: 21900, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/AT_" + \
                     period + "_3_hourly_" + exp + "_4.mat", {'AT': AT_daily[21900:, :, :]})
        
        del AT_daily
        gc.collect()
        
        HI_daily = Heat_index_calcu(TSA_daily, RH2M_daily)
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/HI_" + \
                     period + "_3_hourly_" + exp + "_1.mat", {'HI': HI_daily[:7300, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/HI_" + \
                     period + "_3_hourly_" + exp + "_2.mat", {'HI': HI_daily[7300: 14600, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/HI_" + \
                     period + "_3_hourly_" + exp + "_3.mat", {'HI': HI_daily[14600: 21900, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/HI_" + \
                     period + "_3_hourly_" + exp + "_4.mat", {'HI': HI_daily[21900:, :, :]})
        
        
        del HI_daily
        gc.collect()


        HUMIDEX_daily = Humidex_calcu(TSA_daily, RH2M_daily)
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/HUMIDEX_" + \
                     period + "_3_hourly_" + exp + "_1.mat", {'HUMIDEX': HUMIDEX_daily[:7300, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/HUMIDEX_" + \
                     period + "_3_hourly_" + exp + "_2.mat", {'HUMIDEX': HUMIDEX_daily[7300: 14600, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/HUMIDEX_" + \
                     period + "_3_hourly_" + exp + "_3.mat", {'HUMIDEX': HUMIDEX_daily[14600: 21900, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/HUMIDEX_" + \
                     period + "_3_hourly_" + exp + "_4.mat", {'HUMIDEX': HUMIDEX_daily[21900:, :, :]})
        
        del HUMIDEX_daily
        gc.collect()
        
        
        T_w_daily = Wet_bulb_temperature_calcu(TSA_daily, RH2M_daily)
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_w_" + \
                     period + "_3_hourly_" + exp + "_1.mat", {'T_w': T_w_daily[:7300, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_w_" + \
                     period + "_3_hourly_" + exp + "_2.mat", {'T_w': T_w_daily[7300: 14600, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_w_" + \
                     period + "_3_hourly_" + exp + "_3.mat", {'T_w': T_w_daily[14600: 21900, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_w_" + \
                     period + "_3_hourly_" + exp + "_4.mat", {'T_w': T_w_daily[21900:, :, :]})
        
        del T_w_daily
        gc.collect()
        
    
        THIC_daily = Temperature_humidity_index_for_comfort_calcu(TSA_daily, RH2M_daily)
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/THIC_" + \
                     period + "_3_hourly_" + exp + "_1.mat", {'THIC': THIC_daily[:7300, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/THIC_" + \
                     period + "_3_hourly_" + exp + "_2.mat", {'THIC': THIC_daily[7300: 14600, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/THIC_" + \
                     period + "_3_hourly_" + exp + "_3.mat", {'THIC': THIC_daily[14600: 21900, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/THIC_" + \
                     period + "_3_hourly_" + exp + "_4.mat", {'THIC': THIC_daily[21900:, :, :]})
        
        del THIC_daily
        gc.collect()
        

        THIP_daily = Temperature_humidity_index_for_physiology_calcu(TSA_daily, RH2M_daily)
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/THIP_" + \
                     period + "_3_hourly_" + exp + "_1.mat", {'THIP': THIP_daily[:7300, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/THIP_" + \
                     period + "_3_hourly_" + exp + "_2.mat", {'THIP': THIP_daily[7300: 14600, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/THIP_" + \
                     period + "_3_hourly_" + exp + "_3.mat", {'THIP': THIP_daily[14600: 21900, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/THIP_" + \
                     period + "_3_hourly_" + exp + "_4.mat", {'THIP': THIP_daily[21900:, :, :]})
        
        del THIP_daily
        gc.collect()
        

        DI_daily = Discomfort_index_calcu(TSA_daily, RH2M_daily)
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/DI_" + \
                     period + "_3_hourly_" + exp + "_1.mat", {'DI': DI_daily[:7300, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/DI_" + \
                     period + "_3_hourly_" + exp + "_2.mat", {'DI': DI_daily[7300: 14600, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/DI_" + \
                     period + "_3_hourly_" + exp + "_3.mat", {'DI': DI_daily[14600: 21900, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/DI_" + \
                     period + "_3_hourly_" + exp + "_4.mat", {'DI': DI_daily[21900:, :, :]})
        
        del DI_daily
        gc.collect()
        

        T_s_65_daily = Swamp_temperature_calcu(TSA_daily, RH2M_daily, 65)
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_s_65_" + \
                     period + "_3_hourly_" + exp + "_1.mat", {'T_s_65': T_s_65_daily[:7300, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_s_65_" + \
                     period + "_3_hourly_" + exp + "_2.mat", {'T_s_65': T_s_65_daily[7300: 14600, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_s_65_" + \
                     period + "_3_hourly_" + exp + "_3.mat", {'T_s_65': T_s_65_daily[14600: 21900, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_s_65_" + \
                     period + "_3_hourly_" + exp + "_4.mat", {'T_s_65': T_s_65_daily[21900:, :, :]})
        
        del T_s_65_daily
        gc.collect()

        T_s_80_daily = Swamp_temperature_calcu(TSA_daily, RH2M_daily, 80)
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_s_80_" + \
                     period + "_3_hourly_" + exp + "_1.mat", {'T_s_80': T_s_80_daily[:7300, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_s_80_" + \
                     period + "_3_hourly_" + exp + "_2.mat", {'T_s_80': T_s_80_daily[7300: 14600, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_s_80_" + \
                     period + "_3_hourly_" + exp + "_3.mat", {'T_s_80': T_s_80_daily[14600: 21900, :, :]})
        scio.savemat("/dodrio/scratch/projects/2022_200/project_output/cesm/yi_yao_IRRMIP/after_process/NorESM/post_process/T_s_80_" + \
                     period + "_3_hourly_" + exp + "_4.mat", {'T_s_80': T_s_80_daily[21900:, :, :]})
        
        del T_s_80_daily
        gc.collect()
        

KeyboardInterrupt: 